In [13]:
from dlisio import dlis
from dataclasses import dataclass
import pathlib
import pandas as pd
from pandas import DataFrame
from geofiles.searcher import Searcher
from lisbelo.mnemonicfix import MnemonicFix

In [14]:
root = pathlib.Path('/home/capyba/Documentos/PedroGit/LisBelo/data')
dlis_list = Searcher.SearchDlis(root)

In [15]:
@dataclass
class DlisBelo:

    MergedLogicalFiles: list[dlis.LogicalFile]

    def __init__(self, dlisfile: dlis.PhysicalFile,path) -> None:
        self.path = path
        self.MergedLogicalFiles = []
        self.MergedFrames = []
        self.DataDictList = []

        self.MergeFiles(dlisfile)
        self.GetFrames()
        self.GetOrigins()
        self.MakeDataDict()

    def MergeFiles(self, dlisfile: dlis.PhysicalFile) -> None:
        """
        Junta os arquivos lógicos
        """
        *files, = dlisfile
        self.MergedLogicalFiles.extend(files)

    def GetFrames(self, ):
        """
        Junta os frames
        """
        self.origins = []
        for file in self.MergedLogicalFiles:
            for origin in file.origins:

                self.origins.extend(origin.file_id)

            self.MergedFrames.extend(file.find('FRAME'))

    def GetOrigins(self):
        self.origins = []
        for file in self.MergedLogicalFiles:
            for origin in file.origins:

                self.origins.append(origin.file_id)


    def MakeDataDict(self):
        for frames in self.MergedFrames:
            self.DataDict(frames)

    def DataDict(self, frame:dlis.Frame):

        DataDict = {}
        DataDict['WELL'] = self.origins[0]
        DataDict['FRAME'] = frame.name
        DataDict['CURVES'] = [ch.name for ch in frame.channels]
        DataDict['UNIT'] = [ch.units for ch in frame.channels]

        df = pd.DataFrame(frame.curves(strict=False))
        df = MnemonicFix.IndexToDept(df)
        df = MnemonicFix.gammarename(df)
        
        df['WELL'] = self.origins[0]
        DataDict['DATAFRAME'] = df
        self.DataDictList.append(DataDict)


In [16]:
@dataclass
class DataDict:
    
    def __init__(self,frame:dlis.Frame) -> None:
        pass

In [17]:
class LisPlaza:
    
    DlisFileList:list[DlisBelo]

    def __init__(self,pathlist:list[pathlib.Path]) -> None:
        self.DlisFileList=[]
        self.OpenList(pathlist)

    def OpenList(self,pathlib:list[pathlib.Path]) -> None:
        """
        Se encarrega de abrir e fechar o arquivo
        """
        for path in pathlib:
            # - Abrindo o arquivo
            dl = dlis.load(f'{path}')
            # - Classe custom
            dlbelo = DlisBelo(dl,path)
            # - Aicionando na lista
            self.DlisFileList.append(dlbelo)
            # - Fechando
            dl.close()

    def GammaScan(self):
        """
        Procura em todos os arquivos GR
        """
        GammaData = []
        for file in self.DlisFileList:
            for dataframe in file.DataDictList:
                if 'GR' in dataframe['DATAFRAME'].columns.values:
                    GammaData.append(dataframe)
        
        return GammaData

    def FileNames(self):
        """
        Essa função retorna 
        """
        for file in self.DlisFileList:
            print(file.DataDictList['WELL'])

        

In [18]:
Dlis_Plaza = LisPlaza(dlis_list)

In [19]:
poco_teste = Dlis_Plaza.DlisFileList



In [20]:
gamma1 = Dlis_Plaza.GammaScan()
gamma1

[{'WELL': '2PR  0001  MABRA2PR  0001  MA1',
  'FRAME': '[0,1,FDC/CNL/GR/BOREHOLE-DEPTH/1/.50/F]',
  'CURVES': ['INDEX4110', 'DRHO', 'GR', 'NPHI', 'RHOB', 'CALI'],
  'UNIT': ['ft', None, None, None, None, 'INCHES'],
  'DATAFRAME':       FRAMENO    DEPT        DRHO          GR    NPHI        RHOB        CALI  \
  0           1  3550.5 -999.250000   42.210938 -999.25 -999.250000 -999.250000   
  1           2  3551.0 -999.250000   40.281250 -999.25 -999.250000 -999.250000   
  2           3  3551.5 -999.250000   40.484375 -999.25 -999.250000 -999.250000   
  3           4  3552.0 -999.250000   42.039062 -999.25 -999.250000 -999.250000   
  4           5  3552.5 -999.250000   42.867188 -999.25 -999.250000 -999.250000   
  ...       ...     ...         ...         ...     ...         ...         ...   
  7396     7397  7248.5    0.051880 -999.250000 -999.25    2.673340   12.146484   
  7397     7398  7249.0    0.045410 -999.250000 -999.25    2.663574   12.142578   
  7398     7399  7249.5  

In [23]:
for file in Dlis_Plaza.DlisFileList:
    for logical in file.MergedLogicalFiles:
        print(f"{logical.tools} pertence ao arquivo {file.DataDictList[0]['WELL']}")

[Tool(HDT___.001)] pertence ao arquivo 2PR  0001  MABRA2PR  0001  MA1
[Tool(HDT___.002)] pertence ao arquivo 2PR  0001  MABRA2PR  0001  MA1
[Tool(GEOLIS.003)] pertence ao arquivo 2PR  0001  MABRA2PR  0001  MA1
[Tool(GEOLIS.004)] pertence ao arquivo 2PR  0001  MABRA2PR  0001  MA1
[Tool(GEOLIS.005)] pertence ao arquivo 2PR  0001  MABRA2PR  0001  MA1
[Tool(ISF/ISF/BHC/BHC/GR)] pertence ao arquivo 2PR  0001  MABRA2PR  0001  MA1
[Tool(ISF/BHC/GR)] pertence ao arquivo 2PR  0001  MABRA2PR  0001  MA1
[Tool(FDC/NGS/CNL)] pertence ao arquivo 2PR  0001  MABRA2PR  0001  MA1
[Tool(HDT)] pertence ao arquivo 2PR  0001  MABRA2PR  0001  MA1
[Tool(VSP_W)] pertence ao arquivo 2PR  0001  MABRA2PR  0001  MA1
[Tool(CST_GR)] pertence ao arquivo 2PR  0001  MABRA2PR  0001  MA1
[Tool(FROM__ISS)] pertence ao arquivo 2PR  0001  MABRA2PR  0001  MA1
[Tool(FDC/CNL/GR)] pertence ao arquivo 2PR  0001  MABRA2PR  0001  MA1
[Tool(NGT)] pertence ao arquivo 2PR  0001  MABRA2PR  0001  MA1
[Tool(FROM__FDN)] pertence ao arqui

In [ ]:
nome_teste = gamma1[10]['WELL']
nome_teste.split()

['1RPR', '0001', 'MABRA1RPR', '0001', 'MA1']

In [24]:
import matplotlib.pyplot as plt

class GammaPlot:

    @staticmethod
    def singleGamma(df_: DataFrame, title: str = None, save=False):

        fig,ax = plt.subplots(figsize=(10, 15), dpi=300)      
        ax1 = plt.subplot2grid((1, 1), (0, 0), rowspan=1, colspan=1)
        fig.suptitle(title, fontsize=26)

        ylim = (df_['DEPT'].max(), df_['DEPT'].min())
        ax1.plot('GR', 'DEPT', data=df_)  # Call the data from the df dataframe
        ax1.set_xlabel("Gamma")  # Assign a track title
        ax1.set_xlim(0, 200)  # Change the limits for the curve being plotted
        ax1.set_ylim(ylim)  # Set the depth range
        ax1.grid()  # Display the grid
        if save:
            plt.savefig(f'{title}.png')
            plt.close(fig)
            return


In [25]:
for df in gamma1:
    GammaPlot.singleGamma(df_=df['DATAFRAME'],title=(f"{df['WELL']}"),save=True)